In [1]:
%matplotlib inline

In [2]:
from IPython.display import clear_output
from pathlib import Path
from PIL import Image
from tqdm import tqdm

import glob
import os
import pprint
import random
import shutil
import tarfile
import time
import torch
import torchvision

import matplotlib.pyplot as plt
import numpy as np
import onnxruntime as ort
import torch.nn.functional as F
import torchvision.transforms as T

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def show_image(data, class_list=None):
    image, label = data
    image = image.numpy().transpose((1, 2, 0))
        
    if class_list:
        label = class_list[label]
        
    plt.title(label)
    plt.imshow(image)

In [4]:
def print_training_history(history):
    training_steps = np.array([step for step, _ in history])
    loss = np.array([loss for _, loss in history])

    fig, ax = plt.subplots(figsize=(15, 15))
    ax.plot(training_steps, loss)

    ax.set(xlabel='training_steps', ylabel='loss')
    ax.grid()
    plt.show()

In [5]:
def model_evaluate(model, test_loader):
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        t = time.time()
        for i, (inputs, label) in enumerate(test_loader):
            inputs, label = inputs.to(device), label.to(device)
            outputs = model(inputs)
            
            _, predicted = torch.max(outputs.data, 1)
            print(label.size(0))
            total += label.size(0)
            correct += (predicted == label).sum().item()
            
            t = time.time() - t

            clear_output(wait=True)
            print("Batch     | Time(s)")
            print("-------------------")
            print(f"{i + 1:5d} / {len(test_loader):5d} | {int(t):7d}")

            
    accuracy = 100 * correct / total
            
    print(f'Accuracy of the network on {len(test_loader.dataset)} test images: {accuracy:.1f} %')
    return accuracy

In [6]:
def mixup_data(x, y, alpha=0.1, use_cuda=True):
    # This is code from the authors of the paper
    # https://arxiv.org/abs/1710.09412
    
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        # Note, this means that MixUp does nothing.
        lam = 1
    
    batch_size = x.size()[0]
    
    if use_cuda:
        # This shuffles our indices and sends these indices to the GPU
        index = torch.randperm(batch_size).cuda()
    else:
        # This shuffles our indices without sending them to the GPU
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [7]:
def save_model(model, dataset, epoch, opt, scheduler, loss_history=None):
    
    model_name = model.__class__.__name__
    dataset_name = dataset.__class__.__name__
    t = int(time.time())
    
    if dataset_name:
        ckpt_path = '/workspace/models/' + model_name + '_' + dataset_name + f"_{t}.ckpt"
    else:
        ckpt_path = '/workspace/models/' + model_name + f"_{t}.ckpt"
        
    temp_scale_fn_custom = scheduler._scale_fn_custom
    temp_scale_fn_ref = scheduler._scale_fn_ref

    scheduler._scale_fn_custom = scheduler._scale_fn_ref()

    # remove the reference so there are no more WeakMethod references in the object
    scheduler._scale_fn_ref = None

    #state = scheduler.state_dict()
    #state.pop("_scale_fn_ref")
    
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'scheduler': scheduler.state_dict(),
            'loss_history': loss_history
            }, ckpt_path)
    
    scheduler._scale_fn_custom = temp_scale_fn_custom
    scheduler._scale_fn_ref = temp_scale_fn_ref

    return ckpt_path

In [8]:
def load_model(model, path):
    
    ckpt_dict = torch.load(path)
    
    model.load_state_dict(ckpt_dict['model_state_dict'])
    
    return ckpt_dict

In [9]:
def make_prediction(model, image, transforms, device=None, dataset=None):
    # Image input is assumed to be an image of height, width, channel.
    # Class list has a list of the objects classification.
    
    model.to(device)
    
    image = transforms(image)
    image.to(device)
    if len(image.shape) < 4:
        image = image.unsqueeze(0)
    image = image.to(device)
    
    pred = model(image)
    pred = torch.nn.Softmax(dim=1)(pred)
    pred = pred.squeeze(0)
    confidence = pred
    pred = torch.argmax(pred).item()
    
    final_pred = (dataset.classes[pred], confidence[pred].item()) if dataset else (pred, confidence[pred].item())
    
    return final_pred

In [10]:
def one_batch_train_mixup(model, images, label, use_cuda, alpha=0.1):
    mixed_x, y_a, y_b, lam = mixup_data(images, label, alpha=alpha, use_cuda=use_cuda)
    pred = model(mixed_x)

    loss = mixup_criterion(criterion, pred, y_a, y_b, lam)
    
    return loss

In [11]:
def train(model, criterion, opt, train_dl, test_dl, start_epoch=0, scheduler=None,
          mixup=False, mixup_pct=0.0, use_amp=False, checkpoint=0, loss_history=None,
          use_swa=False, swa_start=0, swa_lr=3e-4, epochs=23, evaluate=True):
    
    # The history of our training, inputs step, loss pair.
    
    history = loss_history if loss_history else []
    
    # Initialize step
    step = 1
    
    # This is the code that allows us to use our GPU to train.
    use_cuda = torch.cuda.is_available()
    device = torch.device('cuda:0' if use_cuda else 'cpu')
    model.to(device)
    
    if use_swa:
        swa_scheduler = torch.optim.swa_utils.SWALR(opt, anneal_strategy="linear", anneal_epochs=5, swa_lr=swa_lr)
        swa_model = torch.optim.swa_utils.AveragedModel(model)
    
    # This controls how many iterations we want to make, which helps control how well our model
    # generalizes.
    total_time = time.time()
    
    if use_amp:
        scaler = torch.cuda.amp.GradScaler()
    
    for epoch in range(start_epoch, epochs):

        # Initializing the time for informational purposes.
        t = time.time()
        batch_start = t

        #This is how we get our inputs and labels from our dataloader.
        for i, (images, label) in enumerate(train_dl):
            # We have to zero out our gradients, because they can get too large.
            opt.zero_grad(set_to_none=True)

            # This sends the inputs and the labels to our device.
            #images, label = images.to(device), label.to(device)
            images = images.to(device)
            label = label.to(device)

            # This is the prediction of our model
            if mixup and random.random() < mixup_pct:
                loss = one_batch_train_mixup(model, images, label, use_cuda=use_cuda, alpha=0.1)
            else:
                pred = model(images)
            
                # Remember loss functions take two inputs, the prediction and the label.
                loss = criterion(pred, label)
            
            # This step gets our gradients.
            if use_amp:
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
            else:
                loss.backward()
                
            if use_swa and epoch > swa_start:
                swa_model.update_parameters(model)
                swa_scheduler.step()
            else:
                # This is how we update our neural network weights. 
                opt.step()
            
            # Append current step and loss.
            history.append((step, loss.item()))
            step += 1

            # Get time Batch training is taking.
            t = time.time() - batch_start

            # Gives a nice representation of how our training is progressing.
            clear_output(wait=True)
            print("Epoch | Batch | Time(s) | Loss")
            print("------------------------------")
            print(f"{epoch + 1:5d} | {i + 1:5d} | {int(t):7d} | {loss:.5f}")
        
        print(f"Finished epoch {epoch + 1}.")
        
        if scheduler:
            scheduler.step()
        
        if checkpoint and ((epoch + 1) % checkpoint == 0):
            save_model(model, train_dl.dataset, epoch, opt, scheduler, loss_history=history)
        
    if use_swa and epochs > swa_start:
        torch.optim.swa_utils.update_bn(train_dl, model, device=device)
    
    total_time = time.time() - total_time
    print(f"Finished Training in {int(total_time)} seconds!")
    if evaluate:
        print("Evaluating")
        model_evaluate(model, test_dl)
        
    print(f"Saving model!")
    save_model(model, train_dl.dataset, epoch, opt, scheduler, loss_history=history)
    
    return history

In [12]:
def save_to_onnx(model, input_shape, path=None):
    bs, c, h, w = input_shape
    
    dummy_input = torch.randn(bs, c, h, w, device='cuda')
    
    model.to('cuda')
    
    input_names = [ "input" ]
    output_names = [ "output" ]
    
    if not path:
        path = model.__class__.__name__ + '.onnx'
    
    torch.onnx.export(model, dummy_input, path, verbose=True, input_names=input_names, output_names=output_names)
    
    return path

In [13]:
class TransformDataset(torch.utils.data.Dataset):

    def __init__(self, dataset, transform, target_transform=None):
        self.dataset = dataset
        self.transform = transform
        self.target_transform=target_transform
        self.name = 'Lakota_Plants'
        
    def __getitem__(self, idx):
        image = self.dataset[idx][0]
        label = self.dataset[idx][1]
        
        if self.transform:
            image = self.transform(image)
            
        if self.target_transform:
            label = self.target_transform(label)

        return image, label

    def __len__(self):
        return len(self.dataset)

# Extract the Training and Test Data

Currently we only have training data, but this will be extracted in the next cell.

In [14]:
# Check if plant_data is made.
if not os.path.isdir('/workspace/data/LAICC_2023/'):
    os.mkdir('/workspace/data/LAICC_2023/')
    
if not os.path.isdir('/workspace/data/LAICC_2023/training'):
    os.mkdir('/workspace/data/LAICC_2023/training')

if not os.path.isdir('/workspace/data/LAICC_2023/test'):
    os.mkdir('/workspace/data/LAICC_2023/test')

#     for f in tqdm(iterable=tar.getmembers(), total=len(tar.getmembers())):
#        tar.extract(member=f)

# Extract data.
if len(glob.glob('/workspace/data/LAICC_2023/training/**/*.jpg')) < 18_000: # Need better integrity check
    with tarfile.open("/workspace/data/LAICC_2023/laicc_2023_training.tar.gz", 'r:gz') as tar:
        for f in tqdm(iterable=tar.getmembers(), total=len(tar.getmembers())):
            tar.extract(member=f, path='/workspace/data/LAICC_2023/training')
        # tar.extractall('/workspace/data/LAICC_2023/training')

if len(glob.glob('/workspace/data/LAICC_2023/test/**/*.jpg')) < 4_000: # Need better integrity check
    with tarfile.open("/workspace/data/LAICC_2023/laicc_2023_test.tar.gz", 'r:gz') as tar:
        for f in tqdm(iterable=tar.getmembers(), total=len(tar.getmembers())):
            tar.extract(member=f, path='/workspace/data/LAICC_2023/test')
        #tar.extractall('/workspace/data/LAICC_2023/test')

# Hyperparameters

These are the variables that you will change to make your model run better or faster.
All of your hyperparameters are below and you will make changes here.

In [15]:
hparams = {
    'lr': 3e-3,
    'num_classes': 7, # Don't change
    # Decide whether you want to add mixup training and how often it is run..
    'mixup': True,
    'mixup_pct': 0.90,
    # Automatic Mixed Precision, should speed up training.
    'use_amp': True,
    # Epochs - how long you want to train
    'epochs': 20,
    'start_epoch': 0,
    # Pin the memory, this should speed up training, but could make the kernel more stable
    'pin_memory': True, # Don't change
    # This is your batch size.
    'bs': 256,
    # Whether or not to use stochastic weight averaging. Setting to true should increase test accuracy.
    # If use_swa is set to False, then swa_start and swa_lr is not used.
    'use_swa': True,
    'swa_start': 17,
    'swa_lr': 5e-3,
    # How many epochs to train before saving model. If set to 0, checkpointing will not be performed.
    'checkpoint': 10,
    # Where your training data is stored.
    'train_root': '/workspace/data/LAICC_2023/training/', # Do not change
    # Where your test data is stored.
    'test_root': '/workspace/data/LAICC_2023/test/', # Do not change
    # This should be a number and how often you should save your data.
    'num_workers': 4,
    # If you want to restart training, change this to the path of the checkpoint you wish to start at.
    'ckpt_path': None, #'/workspace/models/ResNet_Lakota_Plants_1658958130.ckpt',
}

### Transforms

Define your transforms below. The minimal transform input is included below, but you can and should add more.
Select new transforms from [Pytorch Transforms](https://pytorch.org/vision/main/transforms.html#transforms)

In [16]:
transforms = T.Compose(
    [#T.Resize(size=(246, 246)),
     T.CenterCrop([224]),
     T.AugMix(),
     #T.RandomPerspective(),
     T.RandomHorizontalFlip(p=0.5),
     T.RandomVerticalFlip(p=0.5),
     T.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 1)),
     T.RandomRotation(degrees=(0, 10)),
     #T.Grayscale(num_output_channels=3),
     T.ToTensor(),
     T.Normalize(mean=torch.tensor([0.485, 0.456, 0.406]), std=torch.tensor([0.229, 0.224, 0.225])),
    ])

test_transforms = T.Compose(
    [#T.Resize(size=(246, 246)),
     T.CenterCrop([224]),
     #T.Grayscale(num_output_channels=3),
     T.ToTensor(),
     T.Normalize(mean=torch.tensor([0.485, 0.456, 0.406]), std=torch.tensor([0.229, 0.224, 0.225])),
    ])

# Initialize the Dataset

We will initialize our dataset using the transforms above.

In [17]:
# Initialize your training dataset by setting train=True
train_ds = torchvision.datasets.ImageFolder(root=hparams['train_root'], transform=transforms,
                                            target_transform=None, loader=Image.open,
                                            is_valid_file=None)

test_ds = torchvision.datasets.ImageFolder(root=hparams['test_root'], transform=test_transforms,
                                            target_transform=None, loader=Image.open,
                                            is_valid_file=None)

In [18]:
train_ds.__class__.__name__ = 'Lakota_Plants'

In [19]:
# Initialize your dataloader
train_dl = torch.utils.data.DataLoader(train_ds,
                                       batch_size=hparams['bs'],
                                       shuffle=True,
                                       sampler=None,
                                       num_workers=hparams['num_workers'],
                                       persistent_workers=True,
                                       pin_memory=hparams['pin_memory'])
test_dl = torch.utils.data.DataLoader(test_ds,
                                      batch_size=hparams['bs'],
                                      shuffle=False,
                                      sampler=None,
                                      num_workers=hparams['num_workers'],
                                      persistent_workers=False,
                                      pin_memory=hparams['pin_memory'])

In [20]:
# Select your model from the following link:
# https://pytorch.org/vision/stable/models.html#classification

#torchvision.models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
#model = torchvision.models.resnet18(weights="IMAGENET1K_V1")

model = torchvision.models.mobilenet_v3_small(weights=torchvision.models.MobileNet_V3_Small_Weights.DEFAULT)

In [21]:
model.classifier[3]

Linear(in_features=1024, out_features=1000, bias=True)

In [22]:
model.classifier[3] = torch.nn.Linear(model.classifier[3].in_features, len(train_ds.classes))

In [27]:
for name, param in model.named_parameters():
    param.requires_grad = True

In [28]:
# Since we are doing classification, we will always use this loss.

criterion = torch.nn.CrossEntropyLoss(label_smoothing=0)

In [29]:
opt = torch.optim.Adam(model.parameters(), lr=hparams['lr'])
scheduler = torch.optim.lr_scheduler.CyclicLR(opt, base_lr=hparams['lr'], max_lr=10*hparams['lr'],
                                          step_size_up=1000, mode='triangular',
                                          cycle_momentum=False)
#scheduler2 = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.95)

#scheduler = torch.optim.lr_scheduler.ChainedScheduler([scheduler1, scheduler2])

# Optimizers

You can select one of the following optimizers.
For all of these, `params=model.parameters()`

- `torch.optim.Adadelta(params, lr=1, rho=0.9, eps=1e-06, weight_decay=0)`
- `torch.optim.Adagrad(params, lr=0.01, lr_decay=0, weight_decay=0, eps=1e-10)`
- `torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)` use lr=3e-4 or close to it for Adam.
- `torch.optim.AdamW(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)`
- `torch.optim.ASGD(params, lr=0.01, lambd=0.0001, alpha=0.75, t0=1000000.0, weight_decay=0)`
- `torch.optim.RMSprop(params, lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0)`
- `torch.optim.SGD(params, lr=0.001, momentum=0, dampening=0, weight_decay=0, nesterov=False)`

In [30]:
if hparams['ckpt_path']:
    ckpt_dict = torch.load(hparams['ckpt_path'])
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    model.load_state_dict(ckpt_dict['model_state_dict'], strict=False)
    opt.load_state_dict(ckpt_dict['optimizer_state_dict'])
    scheduler.load_state_dict(ckpt_dict['scheduler'])
    hparams['start_epoch'] = ckpt_dict['epoch']
    for state in opt.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.to(device)

    
else:
    ckpt_dict = {
            'epoch': 0,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'scheduler': scheduler.state_dict(),
            'loss_history': None,
    }

In [ ]:
# Call to begin training
# Change the value of epochs
history = train(model, criterion, opt, train_dl, test_dl, start_epoch=hparams['start_epoch'], 
                scheduler=scheduler, mixup=hparams['mixup'], mixup_pct=hparams['mixup_pct'],
                use_amp=hparams['use_amp'], checkpoint=hparams['checkpoint'],
                loss_history=ckpt_dict['loss_history'], use_swa=hparams['use_swa'],
                swa_start=hparams['swa_start'], swa_lr=hparams['swa_lr'],
                epochs=hparams['epochs'], evaluate=True)

Epoch | Batch | Time(s) | Loss
------------------------------
   10 |    28 |     153 | 1.25862


In [ ]:
model_evaluate(model, test_dl)

In [ ]:
print_training_history(history)

In [ ]:
y_true_test = []

for _, labels in tqdm(test_dl):
    labels = torch.flatten(labels)
    labels = [l.item() for l in labels]
    y_true_test.append(labels)

In [ ]:
y_true_test = [item for sublist in y_true_test for item in sublist]

In [ ]:
y_pred = []

model.to('cuda')

for image, _ in tqdm(test_dl):
    
    model.eval()
    
    image = image.to('cuda')
    
    with torch.no_grad():
        preds = torch.nn.Softmax(dim=1)(model(image))
        labels = torch.argmax(preds, dim=1)
        labels = torch.flatten(labels)
        labels = [l.item() for l in labels]
        y_pred.append(labels)

In [ ]:
y_pred = [item for sublist in y_pred for item in sublist]

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
cfm = confusion_matrix(y_true_test, y_pred)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cfm / np.sum(cfm, axis=-1),
                              display_labels=train_ds.classes)
                              #display_labels=[i for i in range(len(base_ds.classes))])


fig, ax = plt.subplots(figsize=(13,13))
disp.plot(ax=ax)
plt.savefig('cfm_resnext_lakota_plant_data.png', format='png')

In [ ]:
# To make a prediction, paste the path to the image below.
# # img_path = '/workspace/data/plant_data/Purple_Coneflower/20220716_230129.jpg'

# img = Image.open(img_path)

In [ ]:
# make_prediction(model.to('cuda'), img, test_transforms, device='cuda', dataset=base_ds)

In [ ]:
acc = model.evaluate(model, test_dl)

In [ ]:
model = model.to("cpu")

In [ ]:
# IF you are satisfied with your models performance, save to ONNX
# You need to change your input shape. It should be of the form
# (batch_size, num_channels, height, width)
# You will get height and width from the transform
# T.Resize(size=(224, 224)), which would mean we would have
input_shape = (10, 3, 224, 224)


path = f'/workspace/models/{model.__class__.__name__}_{cpu}_{acc:.2f}_{int(time.time())}.onnx'

save_to_onnx(model, input_shape, path=path)

In [ ]:
torch_path = f'/workspace/models/{model.__class__.__name__}_{cpu}_{acc:.2f}_{int(time.time())}.pt'

In [ ]:
torch.save(model, torch_path)